In [12]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


path_raw_data = 'data/raw/'
path_processed_data = 'data/processed/'


for split in ['train', 'test']:

    for corpus in tqdm([
        'bo',
        'cl',
        'co',
        'gl',
        'ig',
        'lu'
    ]):

        data = pd.read_csv(
            path_raw_data + f'{split}_r3_{corpus}_top_mentioned_timelines.csv', 
            sep = ';', 
            encoding='utf-8-sig'
            )
        
        data = data[~(data.Texts == 'na')]
        
        if split == 'train':
            
            a
        
        
            
            
        
        
        # data_filtered.to_csv(path_processed_data + f'train_r3_{corpus}_processed.csv')

  0%|          | 0/6 [00:00<?, ?it/s]


NameError: name 'a' is not defined

In [17]:

data

,User_ID,Polarity,Texts
0,r2_bo_1,against,Saudadinhaa dl :/ # Gente que beija com mordid...
1,r2_bo_2,against,PQP ESSE DORAMA É MUITO FOADA(Sassy GoGo(Cheer...
2,r2_bo_5,for,"@This_IsK Hahahaha olha, salada caesar é fácil..."
3,r2_bo_6,against,nao serio pra que # @sabrinalee05_ tô vendo el...
4,r2_bo_7,against,@Chiico_Sant vai passar aqui ou quer que eu vá...
...,...,...,...
558,r2_bo_750,for,3.4- O Centro de Coordenação da Operação está ...
559,r2_bo_751,for,@toko_tkd Indo # @toko_tkd Demorou pra respond...
560,r2_bo_752,for,3.4- O Centro de Coordenação da Operação está ...
561,r2_bo_754,for,O ecoturismo é uma das atividades que podem pr...
